# Binary text classification - IMDB - sequence_Model - bidirectinal_LSTM - word embedding with layer and mask

In [1]:
# Donwload data and create datasets
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

# Listing 11.12 Preparing integer sequence datasets
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=1)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  50.6M      0  0:00:01  0:00:01 --:--:-- 50.6M
Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [7]:
# Listing 11.17 Using an Embedding layer with masking enabled
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 44s 59ms/step - loss: 0.3959 - accuracy: 0.8171 - val_loss: 0.2757 - val_accuracy: 0.8814
Epoch 2/10
625/625 [==============================] - 36s 57ms/step - loss: 0.2253 - accuracy: 0.9148 - val_loss: 0.2840 - val_accuracy: 0.8878
Epoch 3/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1599 - accuracy: 0.9417 - val_loss: 0.2956 - val_accuracy: 0.8910
Epoch 4/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1207 - accuracy: 0.9564 - val_loss: 0.3288 - val_accuracy: 0.8838
Epoch 5/10
625/625 [==============================] - 35s 56ms/step - loss: 0.0856 - accuracy: 0.9696 - val_loss: 0.4231 - val_accuracy: 0.8826
Epoch 6/10
625/625 [==============================] - 34s 54ms/step - loss: 0.0647 - accuracy: 0.9775 - val_loss: 0.4813 - val_accuracy: 0.8666
Epoch 7/10
625/625 [==============================] - 37s 59ms/step - loss: 0.0492 - accuracy: 0.9840 - val_loss: 0.4351 - val_accuracy:

In [8]:
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 26s 30ms/step - loss: 0.2883 - accuracy: 0.8806
Test acc: 0.881
